In [7]:
#[1]   2025-10-6 H=X+JZ
#####   E0, E1  generate ground state, excited state
from qiskit import *
import numpy as np
#### 
n_site=1  
N=n_site 
J=1       ### parameter
step_max=288
t_max=36
dt=t_max/step_max

### simulator and Shots
simulator=Aer.get_backend('qasm_simulator')
n_shots=1000000 #10^6   trial numbers

##### define quantum circuit
qr=QuantumRegister(6,'qr')        ### qr[5]: physical qubit , qr[0]-qr[4]: ancilla qubits
cr=ClassicalRegister(6,'cr')
qc=QuantumCircuit(qr, cr)

### measurement
qc_meas=QuantumCircuit(qr,cr)
qc_meas.measure(qr,cr)

### LIst
time_list_qiskit=[]
magx_list_qiskit=[]
mag_list_qiskit=[]

##### Initial physical state  

#qc.x(qr[5])  ####    E0-qc.x(qr[5]), E1-#qc.x(qr[5]) 
    
#### measurement of X    
qc.h(qr[5])
qc_total=qc.compose(qc_meas)
  #  qc_total=qc+qc_meas  OLD
qc.h(qr[5])
    
job=execute(qc_total, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total)
       
magx=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site):
        obs=obs+(-1)**(int(bit_string[j]))
        if j>=n_site-1: 
            magx=magx+obs*count/n_shots/(n_site)
        else:
            magx=magx
#time_list_qiskit.append(dt*step)
magx_list_qiskit.append(magx)
################################################################################
#### measument of Z
qc_total2=qc.compose(qc_meas)
    
job2=execute(qc_total2, simulator, shots=n_shots)
result2=job2.result()
counts2=result2.get_counts(qc_total2)
      
mag=0
for bit_string, count in counts2.items():
    obs2=0
    for j in range(n_site):
        obs2=obs2+(-1)**(int(bit_string[j]))
        if j>=n_site-1: 
            mag=mag+obs2*count/n_shots/(n_site)
        else:
            mag=mag
    #time_list_qiskit.append(dt*step)
mag_list_qiskit.append(mag)
###################################################
    
print('X=',magx)   ######  X
print('Z=',mag)  #####   Z
print('EE=',magx+J*mag)
print('[0]-results')

    


X= 0.0019540000000000113
Z= 1.0
EE= 1.001954
[0]-results


In [8]:
# [2-1]  the first adaptive twirling operation
J=1
nn=100          ### step of duration
E=1.001954    ### substitute EE in [0]
T1=np.pi/(2*E*(nn))  ### duration
#################################################
qc.h(qr[4])          ### Hadamard transfromation 
qc.s(qr[4])           ### factor i in  ie^{-i\tau{H}}
for i2 in range(nn):   ####  controlled-rotation by Hamiltonian
    qc.rz(0.25*J*T1,qr[5])   
    qc.cx(qr[4],qr[5])
    qc.rz((-0.5)*J*T1,qr[5])
    qc.cx(qr[4],qr[5])
    qc.rz(0.25*J*T1,qr[5])  
    
    qc.rx(0.5*(1)*T1,qr[5]) 
    qc.cz(qr[4],qr[5])
    qc.rx(-1*(1)*T1,qr[5])    
    qc.cz(qr[4],qr[5])
    qc.rx(0.5*(1)*T1,qr[5])
    
    qc.rz(0.25*J*T1,qr[5])   
    qc.cx(qr[4],qr[5])
    qc.rz((-0.5)*J*T1,qr[5])
    qc.cx(qr[4],qr[5])
    qc.rz(0.25*J*T1,qr[5])     
     
    
qc.h(qr[4])   ### Hadamard transformation
  
#############################  measurement of X

qc.h(qr[5])
qc_total=qc.compose(qc_meas)
  #  qc_total=qc+qc_meas  OLD
qc.h(qr[5])

job=execute(qc_total, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total)
       
magx=0
zero=0
temp=0
for bit_string, count in counts.items():
    obs=0
    for j in range(1):
        obs=((-1)**(int(bit_string[0])))*count
        obs=obs*(1-int(bit_string[1]))
        #if j>=n_site-1: 
        temp=(1-(int(bit_string[1])))*count
        magx=magx+obs
        zero=zero+temp
        #else:
           # magx=magx
magx=magx/zero
#time_list_qiskit.append(dt*step)
magx_list_qiskit.append(magx)
######################################## measurement of Z
qc_total2=qc.compose(qc_meas)
    
job2=execute(qc_total2, simulator, shots=n_shots)
result2=job2.result()
counts2=result2.get_counts(qc_total2)
#print(counts2)
    
mag=0
zero=0
temp=0
for bit_string, count in counts2.items():
    obs2=0
    for j in range(1):
        obs2=((-1)**(int(bit_string[0])))*count
        obs2=obs2*(1-int(bit_string[1]))
        temp=(1-(int(bit_string[1])))*count
       #if j>=n_site-1: 
        mag=mag+obs2
        zero=zero+temp    
       # else:
            #mag=mag
    #time_list_qiskit.append(dt*step)
mag=mag/zero
#mag_list_qiskit.append(mag)
###################################################
print('zero=', zero)   #### number of active states 
print('X=',magx)   ######  X
print('Z=',mag)  #####   Z
print('EE=',magx+J*mag)
print('2-1-results')


zero= 782152
X= 0.5650348059569498
Z= 0.7969473964139963
EE= 1.361982202370946
2-1-results


In [9]:
# [2-2]   the second adaptive twirling operation
J=1
nn=100
E=1.3619822  ### substitute EE in 2-1-results
T1=np.pi/(2*E*(nn))     ### duration
#################################################
qc.h(qr[3])              ### Hadamard
qc.s(qr[3])              ### i
for i2 in range(nn):     ### controlled-rotation
    qc.rz(0.25*J*T1,qr[5])   
    qc.cx(qr[3],qr[5])
    qc.rz((-0.5)*J*T1,qr[5])
    qc.cx(qr[3],qr[5])
    qc.rz(0.25*J*T1,qr[5])  
    
    qc.rx(0.5*(1)*T1,qr[5]) 
    qc.cz(qr[3],qr[5])
    qc.rx(-1*(1)*T1,qr[5])    
    qc.cz(qr[3],qr[5])
    qc.rx(0.5*(1)*T1,qr[5])
    
    qc.rz(0.25*J*T1,qr[5])   
    qc.cx(qr[3],qr[5])
    qc.rz((-0.5)*J*T1,qr[5])
    qc.cx(qr[3],qr[5])
    qc.rz(0.25*J*T1,qr[5])     
     
    
qc.h(qr[3])    ### Hadamard 
 
#############################  measurement of X
############################## 
qc.h(qr[5])
qc_total=qc.compose(qc_meas)
  #  qc_total=qc+qc_meas  OLD
qc.h(qr[5])
    

job=execute(qc_total, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total)
       
magx=0
zero=0
temp=0
for bit_string, count in counts.items():
    obs=0
    for j in range(1):
        obs=((-1)**(int(bit_string[0])))*count
        obs=obs*(1-int(bit_string[1]))*(1-int(bit_string[2])) 
        magx=magx+obs
        temp=(1-int(bit_string[1]))*(1-int(bit_string[2]))*count
        zero=zero+temp
        #obs=obs*(1-int(bit_string[1]))*(1-int(bit_string[2]))  2023-1-11 original
        #if j>=n_site-1: 
        ### 2023-1-11 NEW
        ## temp=(1-(int(bit_string[1])))*(1-int(bit_string[2]) 2023-1-11 original
        #####
        #else:
           # magx=magx
magx=magx/zero
#time_list_qiskit.append(dt*step)
magx_list_qiskit.append(magx)
######################################### measurement of Z
qc_total2=qc.compose(qc_meas)
    
job2=execute(qc_total2, simulator, shots=n_shots)
result2=job2.result()
counts2=result2.get_counts(qc_total2)
#print(counts2)
       
mag=0
zero=0
temp=0
for bit_string, count in counts2.items():
    obs2=0
    for j in range(1):
        obs2=((-1)**(int(bit_string[0])))*count
        obs2=obs2*(1-int(bit_string[1]))*(1-int(bit_string[2]))    ### 2023-1-11 NEW
        ###  obs2=obs2*(1-int(bit_string[1]))*(1-int(bit_string[2])) 2023-1-11 original
        temp=(1-int(bit_string[1]))*(1-int(bit_string[2]))*count   ### 2023-1-11 NEW
        ### temp=(1-(int(bit_string[1])))*(1-int(bit_string[2]))*count 2023-1-11 original
       #if j>=n_site-1: 
        mag=mag+obs2
        zero=zero+temp    
       # else:
            #mag=mag
    #time_list_qiskit.append(dt*step)
mag=mag/zero
###################################################
print('zero=', zero)    
print('X=',magx)   ######  X
print('Z=',mag)  #####   Z
print('EE=',magx+J*mag)
print('2-2-results')

zero= 767313
X= 0.7108967559156908
Z= 0.7021971477089531
EE= 1.413093903624644
2-2-results


In [10]:
# [2-3]   the third adaptive twirling operation
J=1
nn=100
E=1.413094      ### substitute EE in 2-2-results
T1=np.pi/(2*E*(nn))
#################################################
qc.h(qr[2])
qc.s(qr[2])
for i2 in range(nn):   ####  controlled-rotation
    qc.rz(0.25*J*T1,qr[5])   
    qc.cx(qr[2],qr[5])
    qc.rz((-0.5)*J*T1,qr[5])
    qc.cx(qr[2],qr[5])
    qc.rz(0.25*J*T1,qr[5])  
    
    qc.rx(0.5*(1)*T1,qr[5]) 
    qc.cz(qr[2],qr[5])
    qc.rx(-1*(1)*T1,qr[5])    
    qc.cz(qr[2],qr[5])
    qc.rx(0.5*(1)*T1,qr[5])
    
    qc.rz(0.25*J*T1,qr[5])   
    qc.cx(qr[2],qr[5])
    qc.rz((-0.5)*J*T1,qr[5])
    qc.cx(qr[2],qr[5])
    qc.rz(0.25*J*T1,qr[5])     
     
    
qc.h(qr[2])

############################## 0-X
qc.h(qr[5])
qc_total=qc.compose(qc_meas)
  #  qc_total=qc+qc_meas  OLD
qc.h(qr[5])
    
job=execute(qc_total, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total)
       
magx=0
zero=0
temp=0
for bit_string, count in counts.items():
    obs=0
    for j in range(1):
        obs=((-1)**(int(bit_string[0])))*count
        obs=obs*(1-int(bit_string[1]))*(1-int(bit_string[2]))*(1-int(bit_string[3]))
        #if j>=n_site-1: 
        temp=(1-(int(bit_string[1])))*(1-(int(bit_string[2])))*(1-int(bit_string[3]))*count
        magx=magx+obs
        zero=zero+temp
        #else:
           # magx=magx
magx=magx/zero
#time_list_qiskit.append(dt*step)
magx_list_qiskit.append(magx)
######################################## 0-Z
qc_total2=qc.compose(qc_meas)
    
job2=execute(qc_total2, simulator, shots=n_shots)
result2=job2.result()
counts2=result2.get_counts(qc_total2)
#print(counts2)
        
mag=0
zero=0
temp=0
for bit_string, count in counts2.items():
    obs2=0
    for j in range(1):
        obs2=((-1)**(int(bit_string[0])))*count
        obs2=obs2*(1-int(bit_string[1]))*(1-int(bit_string[2]))*(1-int(bit_string[3]))
        temp=(1-(int(bit_string[1])))*(1-int(bit_string[2]))*(1-int(bit_string[3]))*count
       #if j>=n_site-1: 
        mag=mag+obs2
        zero=zero+temp    
       # else:
            #mag=mag
    #time_list_qiskit.append(dt*step)
mag=mag/zero
#mag_list_qiskit.append(mag)
###################################################
print('zero=', zero)    
print('X=',magx)   ######  X
print('Z=',mag)  #####   Z
print('EE=',magx+J*mag)
print('2-3-result')



zero= 767074
X= 0.7065699062633592
Z= 0.7064637831552106
EE= 1.4130336894185698
2-3-result


In [11]:
# [2-4]  the fourth adaptive twirling operation
J=1
nn=100
E= 1.41443983  ##### substiteute EE in 2-3-results
T1=np.pi/(2*E*(nn))
#################################################
qc.h(qr[1])
qc.s(qr[1])
for i2 in range(nn):   #### 
    qc.rz(0.25*J*T1,qr[5])   
    qc.cx(qr[1],qr[5])
    qc.rz((-0.5)*J*T1,qr[5])
    qc.cx(qr[1],qr[5])
    qc.rz(0.25*J*T1,qr[5])  
    
    qc.rx(0.5*(1)*T1,qr[5]) 
    qc.cz(qr[1],qr[5])
    qc.rx(-1*(1)*T1,qr[5])    
    qc.cz(qr[1],qr[5])
    qc.rx(0.5*(1)*T1,qr[5])
    
    qc.rz(0.25*J*T1,qr[5])   
    qc.cx(qr[1],qr[5])
    qc.rz((-0.5)*J*T1,qr[5])
    qc.cx(qr[1],qr[5])
    qc.rz(0.25*J*T1,qr[5])     
     
    
qc.h(qr[1])

############################## 0-X
qc.h(qr[5])
qc_total=qc.compose(qc_meas)
  #  qc_total=qc+qc_meas  OLD
qc.h(qr[5])
    

job=execute(qc_total, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total)
       
magx=0
zero=0
temp=0
for bit_string, count in counts.items():
    obs=0
    for j in range(1):
        obs=((-1)**(int(bit_string[0])))*count
        obs=obs*(1-int(bit_string[1]))*(1-int(bit_string[2]))*(1-int(bit_string[3]))*(1-int(bit_string[4]))
        #if j>=n_site-1: 
        temp=(1-(int(bit_string[1])))*(1-(int(bit_string[2])))*(1-int(bit_string[3]))*(1-int(bit_string[4]))*count
        magx=magx+obs
        zero=zero+temp
        #else:
           # magx=magx
magx=magx/zero
#time_list_qiskit.append(dt*step)
#magx_list_qiskit.append(magx)
##################################### 0-Z
qc_total2=qc.compose(qc_meas)
    
job2=execute(qc_total2, simulator, shots=n_shots)
result2=job2.result()
counts2=result2.get_counts(qc_total2)
#print(counts2)
        
mag=0
zero=0
temp=0
for bit_string, count in counts2.items():
    obs2=0
    for j in range(1):
        obs2=((-1)**(int(bit_string[0])))*count
        obs2=obs2*(1-int(bit_string[1]))*(1-int(bit_string[2]))*(1-int(bit_string[3]))*(1-int(bit_string[4]))
        temp=(1-(int(bit_string[1])))*(1-int(bit_string[2]))*(1-int(bit_string[3]))*(1-int(bit_string[4]))*count
       #if j>=n_site-1: 
        mag=mag+obs2
        zero=zero+temp    
       # else:
            #mag=mag
    #time_list_qiskit.append(dt*step)
mag=mag/zero
###################################################
print('zero=', zero)   ### active states   
print('X=',magx)   ######  X
print('Z=',mag)  #####   Z
print('EE=',magx+J*mag)
print('2-4-result')



zero= 766438
X= 0.7081381691042369
Z= 0.7078928758751524
EE= 1.4160310449793894
2-4-result


In [12]:
# [2-5]  the fifth adaptive twirling operation
J=1
nn=100
E=1.416031     ### substitute EE in 2-4-results
T1=np.pi/(2*E*(nn))
#################################################
qc.h(qr[0])
qc.s(qr[0])
for i2 in range(nn):   ####  
    qc.rz(0.25*J*T1,qr[5])   
    qc.cx(qr[0],qr[5])
    qc.rz((-0.5)*J*T1,qr[5])
    qc.cx(qr[0],qr[5])
    qc.rz(0.25*J*T1,qr[5])  
    
    qc.rx(0.5*(1)*T1,qr[5]) 
    qc.cz(qr[0],qr[5])
    qc.rx(-1*(1)*T1,qr[5])    
    qc.cz(qr[0],qr[5])
    qc.rx(0.5*(1)*T1,qr[5])
    
    qc.rz(0.25*J*T1,qr[5])   
    qc.cx(qr[0],qr[5])
    qc.rz((-0.5)*J*T1,qr[5])
    qc.cx(qr[0],qr[5])
    qc.rz(0.25*J*T1,qr[5])     
     
    
qc.h(qr[0])

############################## 0-X
qc.h(qr[5])
qc_total=qc.compose(qc_meas)
  #  qc_total=qc+qc_meas  OLD
qc.h(qr[5])
    

job=execute(qc_total, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total)
       
magx=0
zero=0
temp=0
for bit_string, count in counts.items():
    obs=0
    for j in range(1):
        obs=((-1)**(int(bit_string[0])))*count
        obs=obs*(1-int(bit_string[1]))*(1-int(bit_string[2]))*(1-int(bit_string[3]))*(1-int(bit_string[4]))*(1-int(bit_string[5]))
        #if j>=n_site-1: 
        temp=(1-(int(bit_string[1])))*(1-(int(bit_string[2])))*(1-int(bit_string[3]))*(1-int(bit_string[4]))*(1-int(bit_string[5]))*count
        magx=magx+obs
        zero=zero+temp
        #else:
           # magx=magx
magx=magx/zero
#time_list_qiskit.append(dt*step)
magx_list_qiskit.append(magx)
######################################### 0-Z
qc_total2=qc.compose(qc_meas)
    

job2=execute(qc_total2, simulator, shots=n_shots)
result2=job2.result()
counts2=result2.get_counts(qc_total2)
#print(counts2)
    
       
mag=0
zero=0
temp=0
for bit_string, count in counts2.items():
    obs2=0
    for j in range(1):
        obs2=((-1)**(int(bit_string[0])))*count
        obs2=obs2*(1-int(bit_string[1]))*(1-int(bit_string[2]))*(1-int(bit_string[3]))*(1-int(bit_string[4]))*(1-int(bit_string[5]))
        temp=(1-(int(bit_string[1])))*(1-int(bit_string[2]))*(1-int(bit_string[3]))*(1-int(bit_string[4]))*(1-int(bit_string[5]))*count
       #if j>=n_site-1: 
        mag=mag+obs2
        zero=zero+temp    
       # else:
            #mag=mag
    #time_list_qiskit.append(dt*step)
mag=mag/zero
#mag_list_qiskit.append(mag)
###################################################
print('zero=', zero)    
print('X=',magx)   ######  X
print('Z=',mag)  #####   Z
print('EE=',magx+J*mag)
print('2-5-results')


zero= 766084
X= 0.7079077004205641
Z= 0.7076717435685904
EE= 1.4155794439891545
2-5-results
